# Классификация изображений с помощью Keras (VGG16)

Описание:

Решение задачи классификации образов (кошек и собак) с помощью keras (VGG16, Imagenet). 

Для начала подключим необходимые модули и выведем их версии, в том числе версию python3. 
Также выведем абсолютный путь к папке с python3 (для разработчиков).

Ссылки на сторонние библиотеки:

[matplotlib](https://matplotlib.org/)

[keras](https://keras.io/)

[urllib python3](https://urllib3.readthedocs.io/en/latest/)

[numpy](http://www.numpy.org/)

[OpenCV python3](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_tutorials.html)

Нажмите `Shift+Enter`, чтобы выполнить код в ячейке. Или нажмите `Kernel --> Restart and Run All` для запуска всех ячеек по очереди.

In [ ]:
%matplotlib inline

# Одна из библиотек для работы с нейросетями
import keras

# Архитектура VGG16
from keras.applications.vgg16 import VGG16

# Модуль для работы с изображениями
from keras.preprocessing import image

# Оптимизаторы
from keras import optimizers

# Функции препроцессинга и декодировки предсказания для VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

# Вспомогательный модуль
from keras.utils import np_utils

# Модуль os - функции для работы с операционной системой, не зависящие от используемой операционной системы.
import os

# Garbage collector
import gc

# Библиотека для работы с графиками и изображениями
import matplotlib
from matplotlib import pyplot as plt

# HTTP клиент
import urllib.request

# Класс для работы с файлами формата json
import json

# Полезная утилит показа progress-bar
from tqdm import tqdm_notebook

# Модуль из OpenCV. Здесь мы применяем resize для определения битых картинок
import cv2

# Библиотека для быстрой работы с многомерными массивами
import numpy as np

# Модуль, который обеспечивает доступ к некоторым переменным и функциям,
# взаимодействующим с интерпретатором python.
import sys

# Модуль для доступа к логину и паролю пользователя
import getpass

# Модуль для работы с файлами формата ZIP
import zipfile

# Модуль glob находит все пути, совпадающие с заданным шаблоном в соответствии с правилами, используемыми оболочкой Unix.
import glob

usrname = getpass.getuser()

print('Версия python3:', sys.version)
print('Версия keras:', keras.__version__)
print('Версия matplotlib:', matplotlib.__version__)
print('Версия urlib.request', urllib.request.__version__)
print('Версия numpy', np.__version__)
print('Версия OpenCV:', cv2.__version__)
print('Абсолютный путь к папке python3:', sys.executable)
print('Имя пользователя', usrname)

Теперь скачаем данные. Это размеченные картинки кошек и собак с соревнования Kaggle. Получить доступ к ним с одноименного сайта проблематично. К счастью, на сайте Майкрософт эти данные тоже есть, поэтому скачивать будем оттуда. Обучение нейросети на практике занимает от одного дня до недели. Однако в Keras уже есть предобученная модель, которую можно использовать.

In [ ]:
%%time

path_to_data_no_split = os.getcwd() + '/data_no_split/'
path_to_zip_file = path_to_data_no_split + 'cat_dog_images.zip'    

# Создаем директорию, если таковая отсутствует
if not os.path.exists(path_to_data_no_split):
    os.makedirs(path_to_data_no_split)

# Скачивание данных
url = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'
if not os.path.exists(path_to_zip_file):
    urllib.request.urlretrieve(url, path_to_zip_file)

# Распаковка архива
zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
if not os.path.exists(path_to_data_no_split+'PetImages/'):
    zip_ref.extractall(path_to_data_no_split)
zip_ref.close()

gc.collect

Теперь необходимо проверить картинки на качество: некоторые файлы могу быть "битыми". Для этого определим функцию `check_class_num`. В ней мы проверяем, сколько изображений можно использовать для обучения, сквозной проверки и теста. Заодно получим список имен файлов, которые можно использовать.

In [ ]:
# Обозначения классов
classes = ["dog", "cat"]
num_classes = len(classes)

# Размерность изображения
img_width, img_height = 224, 224

# Посчитать число небитых картинок
def check_class_num(img_begin=0, img_count=100):
    folders = ["Dog", "Cat"]
    cat_dog_num = {folders[0] : 0, folders[1] : 0}
    cat_dog_list = {folders[0] : [], folders[1] : []}
    
    for fld in folders:
        index = folders.index(fld)
        path = os.path.join(path_to_data_no_split+'PetImages/', fld, '*g')
        files = glob.glob(path)
        i = 0
        
        for fl in tqdm_notebook(files[img_begin:img_begin+img_count]):
            flbase = os.path.basename(fl)
            try:
                img = cv2.imread(fl)
                resized = cv2.resize(img, (img_width, img_height), cv2.INTER_LINEAR)
            except:
                continue
            cat_dog_num[fld] += 1
            cat_dog_list[fld] += [fl]
            
    gc.collect
    return cat_dog_num, cat_dog_list

In [ ]:
cat_dog_num, cat_dog_list = check_class_num(img_begin=0, img_count=12500)

Мы получили список файлов, которые можно читать и использовать для классификации. Сейчас создадим модель нейронной сети VGG-16 с весами. В Keras уже есть натренированная модель на основе данных ImageNet. Модель решает задачу классификации для 1000 классов. Эта модель умеет классифицировать гораздо более широкий диапазон объектов, но все интересно, как она будет работать на наших изображениях.

In [ ]:
# https://keras.io/applications/
model = VGG16(weights='imagenet')

Модель загрузили, теперь попробуем ее применить. Для этого определим функцию `predict_show`.

In [ ]:
def predict_show(folder='Dog', count=5):
    for img_path in cat_dog_list[folder][:count]:
        img = image.load_img(img_path, target_size=(img_width, img_height))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        preds = model.predict(x)

        fig, ax = plt.subplots()
        ax.imshow(img)
        ax.axis('off')  # Очистка от разметки осей
        # Формат: (имя_класса, вероятность) top=1 => максимальная уверенность в классе среди 1000 классов.
        print('Предсказание:', decode_predictions(preds, top=1)[0][0][1:3])
        plt.show()

Применим ее для изображений собак.

In [ ]:
predict_show(folder='Dog', count=5)

Применим ее для изображений кошек.

In [ ]:
predict_show(folder='Cat', count=5)

Теперь ясно, что сейчас очень просто начать пользоваться нейросетью: достаточно установить пакеты и написать несложный код. Однако существуют другие задачи, для которых решение приходится писать самому. В блокноте 2_notebook_VGG16_cat_dog_train показана архитектура VGG16, ее модификация для бинарной классификации (собака, кошка), обучение и тесты. Однако запускать код на маломощных машинах строго не рекомендуется, все необходимые файлы либо скачиваются из интернета, либо находятся в директории. Таким образом, в блокноте 3_notebook_VGG16_cat_dog_using используется предобученная модель.